In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export base
%nbdev_default_class_level 3

Cells will be exported to geomechy.base,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import pandas as pd
import numpy as np
import scipy
import torch

from geomechy.utils import *
from geomechy.shape import getElemetType

# Base
> all the base classes

## Base Dict

In [ ]:
%nbdev_export
class ItemDict(dict):
    
    def add(self, Id, item):
        assert isinstance(Id,int)
        self[Id] = item
        
    def get(self, Ids):
        return (self[Ids] if isinstance(Ids,int) else [self[Id] for Id in Ids] 
                if isinstance(Ids,list) else "NAN")

    def getData(self, Ids):
        return {Id:self[Id] for Id in Ids}

## Base Properties

In [ ]:
%nbdev_export
class Properties:
    
    def __init__ (self, props={}):

        for key in props.keys():
            setattr(self, key, props[key])
    
    def __str__ (self):
        att_prop  = ''
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
                
            att_prop += 'Attribute: ' + att + '\n'
            att_prop += str(getattr(self,att)) + '\n'

        return att_prop
    
    def __iter__ ( self ):
        propsList = []
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
            propsList.append((att, getattr(self,att)))

        return iter(propsList)
    
    def store (self, key, val):
        setattr(self, key, val)

## Base Constitutive

In [ ]:
%nbdev_export
class BaseConstitutive:
    
    def __init__(self, params):
        for name,val in params.items():
            dim = eval(val["dim"])
            value = val["value"]
            setattr(self, name, value*dim)

        self.current = {"i":0}
        self.iIter   = -1
     
    def __str__ (self):
        return f'current iter: {self.iIter}'

    def setCurrentParameter(self, name, val):
        self.current[name]=val
        
    def getCurrentParameter(self, name):
        return self.current[name]
    
    def setIter(self, iIter):
        self.iIter = iIter
        
    def commitHistory(self):
        if self.iIter == -1: 
            self.history = pd.DataFrame(self.current, index=[0])
        else:
            self.history = self.history.append(self.current, ignore_index=True)
        
        self.setIter(0)
        for key, val in self.current.items():
            self.current[key] = 0.

    def getHistoryParameter(self, name, k):
        return self.history.loc[k,name]

## Base Element

In [ ]:
%nbdev_export
class BaseElement(dict):
    
    def __init__ (self, Id, nodes, elems, group, shape, mat, consti):
        
        self.Id          = Id  
        self.nodes       = list(np.sort(np.array(elems[Id]["Nodes"] )))
        self.nodesCoords = nodes
        self.coords      = np.array([self.nodesCoords[node] for node in self.nodes])
        self.elemType    = getElemetType(self.coords)
        self.Le          = elems[Id]["Le"]
        self.gp          = shape["gp"]
        self.we          = shape["we"]
        self.N           = shape["N"]
        self.dN          = shape["dN"]
        self.group       = group
        self.mat         = mat
        self.consti      = consti
        
        self.iIter = -1
        self.history_nodes = pd.DataFrame({"iter":[0]})
        self.history_gps   = pd.DataFrame({"iter":[0]})
        self.current = {"iter":0}
        
        self.add(self.Id, self.nodes)
        
    def add(self, Id, item):
        self[Id] = item 
        
    def setIter(self, iIter):
        self.iIter = iIter
        
    def setHistoryNodesParameter(self, name, val):
        if self.iIter == -1:
            self.history_nodes[name]=val
            
    def setHistoryGaussParameter(self, name, val):
        if self.iIter == -1:
            self.history_gps[name]=val
        
    def setCurrentParameter(self, name, val):
        if self.iIter == -1:
            self.current[name]=val
    
    def getHistoryNodesParameter(self, iIter, name):
        return self.history_nodes.loc[iIter,name]
    
    def getHistoryGaussParameter(self, iIter, name):
        return self.history_gps.loc[iIter,name]
    
    def getCurrentParameter(self, name):
        return self.current[name]
        
    def commitHistory(self):
        self.setIter(0)
        self.history_nodes = self.history_nodes.append(self.current, ignore_index=True)
        self.history_gps   = self.history_gps.append(self.current, ignore_index=True)
        for key, val in self.current.items():
            self.current[key] = 0.

## Base Linear System Solver

In [ ]:
%nbdev_export
class BaseLinearSolver:
    def __init__(self, A, b, xo=np.array([]), tol=1e-10, maxIter=10000):
        self.device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.A       = A
        self.b       = b
        self.xo      = xo
        self.tol     = tol
        self.maxIter = maxIter
        
        self.A_sparsity = 1.0-np.count_nonzero(self.A.todense())/self.A.todense().size
        self.b_sparsity = 1.0-np.count_nonzero(self.b)/self.b.size
        
        if not self.xo.any():
            self.xo = 10*np.random.randn(self.A.todense().shape[0],1)
        
        if self.device == torch.device(type='cuda'):
            self.xo = torch.from_numpy(self.xo).type(torch.float64).to(self.device)
            self.b  = torch.from_numpy(self.b).type(torch.float64).to(self.device)
            self.A  = torch.from_numpy(self.A.todense()).type(torch.float64).to(self.device)
    
    def __str__ (self):
        return f'Solver device: {self.device}'

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_constitutive.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 05_Solvers.ipynb.
Converted 06_materials.ipynb.
Converted 07_fem.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
Converted test.ipynb.
